In [1]:
using Statistics: mean

import FFTW

using Interact
using PyPlot

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/05a54a8cd5b6d9c9183d90a9a0f9abc6032d3417-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/05a54a8cd5b6d9c9183d90a9a0f9abc6032d3417-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [ ]:
function solve_poisson_3d_mbc(f, Lx, Ly, Lz, wavenumbers)
    Nx, Ny, Nz = size(f)  # Number of grid points (excluding the periodic end point).

    # Forward transform the real-valued source term.
    fh = FFTW.dct(FFTW.rfft(f, [1, 2]), 3)

    # Wavenumber indices.    
    l1 = 0:Int(Nx/2)
    l2 = Int(-Nx/2 + 1):-1
    m1 = 0:Int(Ny/2)
    m2 = Int(-Ny/2 + 1):-1
    n1 = 0:Int(Nz/2)
    n2 = Int(-Nz/2 + 1):-1

    if wavenumbers == :second_order
        Δx = Lx/Nx
        Δy = Ly/Ny
        Δz = Lz/Nz
        
        kx² = reshape((4/Δx^2) .* sin.( (π/Nx) .* cat(l1, l2, dims=1)).^2, (Nx, 1, 1))
        ky² = reshape((4/Δy^2) .* sin.( (π/Ny) .* cat(m1, m2, dims=1)).^2, (1, Ny, 1))
        # kz² = reshape((2/Δz^2) .* (cos.( (π/Nz) .* cat(n1, n2, dims=1)) .- 1), (1, 1, Nz))
        kz² = reshape((2/Δz^2) .* (cos.( (π/Nz) .* (1:(Nz-1))) .- 1), (1, 1, Nz))
        
        k² = @. -kx² - ky² + kz²
        ϕh = fh ./ k²[1:Int(Nx/2 + 1), :, :]
    elseif wavenumbers == :analytic
        kx = reshape((2π/Lx) * cat(l1, l2, dims=1), (Nx, 1, 1))
        ky = reshape((2π/Ly) * cat(m1, m2, dims=1), (1, Ny, 1))
        kz = reshape((1π/Lz) * cat(n1, n2, dims=1), (1, 1, Nz))
        
        k² = @. kx^2 + ky^2 + kz^2
        ϕh = - fh ./ k²[1:Int(Nx/2 + 1), :, :]
    end

    # Setting the DC/zero Fourier component to zero.
    ϕh[1, 1, 1] = 0

    # Take the inverse transform of the solution's Fourier coefficients.
    ϕ = FFTW.irfft(FFTW.idct(ϕh, 3), Nx, [1, 2])
end

In [ ]:
A = rand(4,4,4)

In [ ]:
N = 8
A = rand(N, N, N)
Ã1 = FFTW.dct(FFTW.rfft(A, [1, 2]), 3)
Ã2 = FFTW.rfft(FFTW.dct(A, 3), [1, 2])
Ã1 ≈ Ã2

In [ ]:
N = 8
A = rand(N, N, N)

Ã1 = FFTW.dct(FFTW.rfft(A, [1, 2]), 3)
Ã2 = FFTW.rfft(FFTW.dct(A, 3), [1, 2])

A11 = FFTW.irfft(FFTW.idct(Ã1, 3), N, [1, 2])
A12 = FFTW.idct(FFTW.irfft(Ã1, N, [1, 2]), 3)
A21 = FFTW.irfft(FFTW.idct(Ã2, 3), N, [1, 2])
A22 = FFTW.idct(FFTW.irfft(Ã2, N, [1, 2]), 3)
A ≈ A11 && A ≈ A12 && A ≈ A21 && A ≈ A22

In [ ]:
Lx, Ly, Lz = 8, 8, 8              # Domain size.
Nx, Ny, Nz = 64, 64, 64           # Number of grid points.
Δx, Δy, Δz = Lx/Nx, Ly/Ny, Lz/Nz  # Grid spacing.

# Grid point locations.
x = Δx * (0:(Nx-1));
y = Δy * (0:(Ny-1));
z = Δz * (0:(Nz-1));

# Primed coordinates to easily calculate a Gaussian centered at
# (Lx/2, Ly/2).
x′ = reshape(x .- Lx/2, (Nx, 1, 1))
y′ = reshape(y .- Ly/2, (1, Ny, 1))
z′ = reshape(z .- Lz/2, (1, 1, Nz))

f = @. 4 * (x′^2 + y′^2 - 1 - (π/Lz)^2) * cos((2π/Lz) * z′) * exp(-(x′^2 + y′^2))  # Source term
f .= f .- mean(f)  # Ensure that source term integrates to zero.

ϕa = @. cos((2π/Lz) * z′) * exp(-(x′^2 + y′^2))  # Analytic solution

ϕs = solve_poisson_3d_mbc(f, Lx, Ly, Lz)

In [ ]:
# @show size(x)
# @show size(y)
# @show size(f)
# @show minimum(f)
# @show maximum(f)
fig = figure()
@manipulate for n in 1:Nz
    withfig(fig) do
        # PyPlot.contourf(x, y, f[:, :, n], levels=20, vmin=-6, vmax=0.5); PyPlot.colorbar();
        PyPlot.pcolormesh(x, y, f[:, :, n], cmap="bwr", vmin=-4, vmax=4); PyPlot.colorbar();
    end
end

In [ ]:
@show minimum(ϕs)
@show maximum(ϕs)
@show minimum(ϕa)
@show maximum(ϕa)

In [ ]:
fig = figure()
@manipulate for n in 1:Nz
    withfig(fig) do
        # PyPlot.contourf(x, y, f[:, :, n], levels=20, vmin=-6, vmax=0.5); PyPlot.colorbar();
        PyPlot.pcolormesh(x, y, ϕs[:, :, n] - ϕa[:, :, n], vmin=-0.5, vmax=0.5, cmap="bwr"); PyPlot.colorbar();
    end
end

In [ ]:
mean(abs.(ϕs - ϕa))

In [ ]:
Ns = [8, 16, 32, 64, 128, 256]
errors = []
for N in Ns
    Lx, Ly, Lz = 8, 8, 8              # Domain size.
    Nx, Ny, Nz = N, N, N              # Number of grid points.
    Δx, Δy, Δz = Lx/Nx, Ly/Ny, Lz/Nz  # Grid spacing.

    # Grid point locations.
    x = Δx * (0:(Nx-1));
    y = Δy * (0:(Ny-1));
    z = Δz * (0:(Nz-1));

    # Primed coordinates to easily calculate a Gaussian centered at
    # (Lx/2, Ly/2).
    x′ = reshape(x .- Lx/2, (Nx, 1, 1))
    y′ = reshape(y .- Ly/2, (1, Ny, 1))
    z′ = reshape(z .- Lz/2, (1, 1, Nz))

    f = @. 4 * (x′^2 + y′^2 - 1 - (π/Lz)^2) * cos((2π/Lz) * z′) * exp(-(x′^2 + y′^2))  # Source term
    f .= f .- mean(f)  # Ensure that source term integrates to zero.

    ϕa = @. cos((2π/Lz) * z′) * exp(-(x′^2 + y′^2))  # Analytic solution

    ϕs = solve_poisson_3d_mbc(f, Lx, Ly, Lz, :second_order)
    
    @show N
    @show minimum(ϕs)
    @show maximum(ϕs)
    @show minimum(ϕa)
    @show maximum(ϕa)
    @show mean_error = mean(abs.(ϕs - ϕa))
    append!(errors, mean_error)
end

In [ ]:
PyPlot.loglog(Ns, errors, linestyle="--", marker="o");
slope = (log10(errors[end]) - log10(errors[1])) / (log10(Ns[end]) - log10(Ns[1]));
PyPlot.xlabel("Nx = Ny = Nz = N")
PyPlot.ylabel("mean(abs(error))")
@show slope;

In [ ]:
Ns = [8, 16, 32, 64, 128, 256]
errors = []
for N in Ns
    Lx, Ly, Lz = 8, 8, 8              # Domain size.
    Nx, Ny, Nz = N, N, N              # Number of grid points.
    Δx, Δy, Δz = Lx/Nx, Ly/Ny, Lz/Nz  # Grid spacing.

    # Grid point locations.
    x = Δx * (0:(Nx-1));
    y = Δy * (0:(Ny-1));
    z = Δz * (0:(Nz-1));

    # Primed coordinates to easily calculate a Gaussian centered at
    # (Lx/2, Ly/2).
    x′ = reshape(x .- Lx/2, (Nx, 1, 1))
    y′ = reshape(y .- Ly/2, (1, Ny, 1))
    z′ = reshape(z .- Lz/2, (1, 1, Nz))

    f = @. ( 4*(x′^2 + y′^2 - 1) - (π/(2*Lz))^2) * cos((π/2*Lz) * z′) * exp(-(x′^2 + y′^2))  # Source term
    f .= f .- mean(f)  # Ensure that source term integrates to zero.

    ϕa = @. cos((π/(2*Lz)) * z′) * exp(-(x′^2 + y′^2))  # Analytic solution

    ϕs = solve_poisson_3d_mbc(f, Lx, Ly, Lz, :analytic)
    ϕs .= ϕs .- minimum(ϕs)
    
    @show N
    @show minimum(ϕs)
    @show maximum(ϕs)
    @show minimum(ϕa)
    @show maximum(ϕa)
    @show mean_error = mean(abs.(ϕs - ϕa))
    append!(errors, mean_error)
end

In [ ]:
@show minimum(ϕs)
@show maximum(ϕs)
@show minimum(ϕa)
@show maximum(ϕa)
ϕs .= ϕs .- minimum(ϕs)
 @show mean(abs.(ϕs - ϕa))

In [ ]:
fig = figure()
@manipulate for n in 1:Nz
    withfig(fig) do
        # PyPlot.contourf(x, y, f[:, :, n], levels=20, vmin=-6, vmax=0.5); PyPlot.colorbar();
        PyPlot.pcolormesh(x, y, ϕs[:, :, n], vmin=-0.5, vmax=0.5, cmap="bwr"); PyPlot.colorbar();
    end
end

In [ ]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

function laplacian(f)
    Nx, Ny, Nz = size(f)
    ∇²f = zeros(Nx, Ny, Nz)
#     for k in 2:(Nz-1), j in 1:Ny, i in 1:Nx
#        ∇²f[i, j, k] = f[incmod1(i, Nx), j, k] + f[decmod1(i, Nx), j, k] + f[i, incmod1(j, Ny), k] + f[i, decmod1(j, Ny), k] + f[i, j, k+1] + f[i, j, k-1] - 6*f[i, j, k]
#     end
    for k in 1:Nz, j in 1:Ny, i in 1:Nx
       ∇²f[i, j, k] = f[incmod1(i, Nx), j, k] + f[decmod1(i, Nx), j, k] + f[i, incmod1(j, Ny), k] + f[i, decmod1(j, Ny), k] + f[i, j, incmod1(k, Nz)] + f[i, j, decmod1(k, Nz)] - 6*f[i, j, k]
    end
    ∇²f
end

In [ ]:
l1 = 0:Int(Nx/2)
l2 = Int(-Nx/2 + 1):-1
m1 = 0:Int(Ny/2)
m2 = Int(-Ny/2 + 1):-1
n1 = 0:Int(Nz/2)
n2 = Int(-Nz/2 + 1):-1

Δx = Lx/Nx
Δy = Ly/Ny
Δz = Lz/Nz

kz² = reshape((2/Δz^2) .* (cos.( (π/Nz) .* cat(n1, n2, dims=1)) .- 1), (1, 1, Nz))

In [ ]:
Nx, Ny, Nz = 10, 10, 10
ff = rand(Nx, Ny, Nz)
ff[:, :, 1] = ff[:, :, 2]
ff[:, :, end] .= ff[:, :, end-1]
ff .= ff .- mean(ff)
ff

In [ ]:
ϕr = solve_poisson_3d_mbc(ff, Nx, Ny, Nz, :second_order)

In [ ]:
ffr = laplacian(ϕr)